In [1]:
import pandas as pd
import json
import sqlite3
import numpy as np

# This block simulates your three source files
# In a real scenario, you would use pd.read_csv(), pd.read_json(), etc.

In [8]:
import pandas as pd

# Load the transactional data
df_orders = pd.read_csv('orders.csv')

# Convert order_date to datetime to help with future questions (Quarter analysis)
df_orders['order_date'] = pd.to_datetime(df_orders['order_date'])

print("Orders loaded. Shape:", df_orders.shape)
df_orders.head()


Orders loaded. Shape: (10000, 6)


/tmp/ipython-input-2115796356.py:7: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_orders['order_date'] = pd.to_datetime(df_orders['order_date'])


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,2023-02-18,842.97,New Foods Chinese
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian


In [9]:
import json

# Load user master data
df_users = pd.read_json('users.json')

print("Users loaded. Shape:", df_users.shape)
df_users.head()

Users loaded. Shape: (3000, 4)


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [10]:
import sqlite3

# Create a connection to an in-memory database
conn = sqlite3.connect(':memory:')

# Read and execute the SQL file
with open('restaurants.sql', 'r') as file:
    sql_script = file.read()
    conn.executescript(sql_script)

# Load the table into a Pandas DataFrame
df_restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)

print("Restaurants loaded from SQL. Shape:", df_restaurants.shape)
df_restaurants.head()

Restaurants loaded from SQL. Shape: (500, 4)


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [11]:
# 1. Join Orders with Users
merged_df = pd.merge(df_orders, df_users, on='user_id', how='left')

# 2. Join the result with Restaurants
final_df = pd.merge(merged_df, df_restaurants, on='restaurant_id', how='left')

print("Final dataset merged. Total rows:", len(final_df))

Final dataset merged. Total rows: 10000


In [12]:
# Save to CSV as required
final_df.to_csv('final_food_delivery_dataset.csv', index=False)
print("File 'final_food_delivery_dataset.csv' has been created.")

File 'final_food_delivery_dataset.csv' has been created.


In [20]:
import pandas as pd

# Load the merged dataset
df = pd.read_csv('final_food_delivery_dataset.csv')
df['order_date'] = pd.to_datetime(df['order_date'])

print("--- HACKATHON ANALYSIS START ---")

# 1. Highest total revenue from Gold members by city
gold_df = df[df['membership'] == 'Gold']
city_rev = gold_df.groupby('city')['total_amount'].sum()
print(f"1. City with highest Gold revenue: {city_rev.idxmax()}")

# 2. Highest Average Order Value (AOV) by cuisine
cuisine_aov = df.groupby('cuisine')['total_amount'].mean()
print(f"2. Cuisine with highest AOV: {cuisine_aov.idxmax()}")

# 3. Distinct users with total spend > ₹1000
user_spend = df.groupby('user_id')['total_amount'].sum()
print(f"3. Distinct users spending > ₹1000: {(user_spend > 1000).sum()}")

# 4. Rating range with highest total revenue
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0-3.5', '3.6-4.0', '4.1-4.5', '4.6-5.0']
df['rating_range'] = pd.cut(df['rating'], bins=bins, labels=labels, include_lowest=True)
rating_rev = df.groupby('rating_range', observed=True)['total_amount'].sum()
print(f"4. Highest revenue rating range: {rating_rev.idxmax()}")

# 5. Highest Gold Average Order Value (AOV) by city
gold_city_aov = gold_df.groupby('city')['total_amount'].mean()
print(f"5. City with highest Gold AOV: {gold_city_aov.idxmax()}")

# 6. Cuisine with lowest distinct restaurants
cuisine_stats = df.groupby('cuisine').agg(distinct_restaurants=('restaurant_id', 'nunique'))
print(f"6. Cuisine with lowest distinct restaurants: {cuisine_stats['distinct_restaurants'].idxmin()}")

# 7. Percentage of total orders by Gold members
gold_pct = (len(gold_df) / len(df)) * 100
print(f"7. Percentage of Gold orders: {round(gold_pct)}%")

# 8. Restaurant with highest AOV but < 20 total orders (Checking MCQ Options)
options_q8 = ['Grand Cafe Punjabi', 'Grand Restaurant South Indian', 'Ruchi Mess Multicuisine', 'Ruchi Foods Chinese']
rest_stats = df[df['restaurant_name_x'].isin(options_q8)].groupby('restaurant_name_x').agg(
    aov=('total_amount', 'mean'),
    count=('order_id', 'count')
)
filtered_q8 = rest_stats[rest_stats['count'] < 20]
print(f"8. Highest AOV Restaurant (<20 orders): {filtered_q8['aov'].idxmax()}")

# 9. Highest revenue combination (Among MCQ Options)
comb_options = [('Gold', 'Indian'), ('Gold', 'Italian'), ('Regular', 'Indian'), ('Regular', 'Chinese')]
comb_results = {f"{m} + {c}": df[(df['membership']==m) & (df['cuisine']==c)]['total_amount'].sum() for m, c in comb_options}
print(f"9. Highest revenue combination: {max(comb_results, key=comb_results.get)}")

# 10. Highest revenue quarter
df['quarter'] = df['order_date'].dt.quarter
quarter_map = {1: 'Q1 (Jan–Mar)', 2: 'Q2 (Apr–Jun)', 3: 'Q3 (Jul–Sep)', 4: 'Q4 (Oct–Dec)'}
print(f"10. Highest revenue quarter: {quarter_map[df.groupby('quarter')['total_amount'].sum().idxmax()]}")

print("\n--- NUMERICAL ANSWERS ---")
print(f"Total orders by Gold members: {len(gold_df)}")
print(f"Total revenue Hyderabad: {round(df[df['city'] == 'Hyderabad']['total_amount'].sum())}")
print(f"Distinct users: {df['user_id'].nunique()}")
print(f"Average order value Gold: {round(gold_df['total_amount'].mean(), 2)}")
print(f"Orders with rating >= 4.5: {len(df[df['rating'] >= 4.5])}")
top_gold_city = city_rev.idxmax()
print(f"Orders in top Gold city ({top_gold_city}): {len(gold_df[gold_df['city'] == top_gold_city])}")

--- HACKATHON ANALYSIS START ---
1. City with highest Gold revenue: Chennai
2. Cuisine with highest AOV: Mexican
3. Distinct users spending > ₹1000: 2544
4. Highest revenue rating range: 4.6-5.0
5. City with highest Gold AOV: Chennai
6. Cuisine with lowest distinct restaurants: Chinese
7. Percentage of Gold orders: 50%
8. Highest AOV Restaurant (<20 orders): Ruchi Foods Chinese
9. Highest revenue combination: Gold + Italian
10. Highest revenue quarter: Q3 (Jul–Sep)

--- NUMERICAL ANSWERS ---
Total orders by Gold members: 4987
Total revenue Hyderabad: 1889367
Distinct users: 2883
Average order value Gold: 797.15
Orders with rating >= 4.5: 3374
Orders in top Gold city (Chennai): 1337
